In [0]:
from google.colab import drive
drive.mount('/Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /Drive


In [0]:
# Load model checkpoints
urls = ["https://www.dropbox.com/s/ud0xpk0aageatmt/faces-BA-100.h5?dl=0", "https://www.dropbox.com/s/mffhew71g0ecvjg/faces-AB-100.h5?dl=0", "https://www.dropbox.com/s/u12v9c3kawbl33s/faces-100.h5?dl=0"]

def download_from_dropbox(urls):
    for url in urls:
      url = url.split("?")[0].replace("dropbox", "dl.dropboxusercontent")
      os.system(f'wget {url}')
  
download_from_dropbox(urls)

## Imports

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-i188p13c
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-i188p13c
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=6fabe3ed5a4c4e027fc4712049a5eef6c4d4c3a6579814d7e01b242c28be2e9b
  Stored in directory: /tmp/pip-ephem-wheel-cache-681o5i2u/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Lambda, Add
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu, tanh
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import backend as K
from glob import glob
import matplotlib.pyplot as plt
import datetime
import scipy
import scipy.misc
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
import os
import sys
import time
import copy
import math
import cv2

np.random.seed(42)


Using TensorFlow backend.


## Get Dataset

In [0]:
from tqdm import tqdm

def download():
  url = 'https://dl.dropboxusercontent.com/s/1i2h7orwm2cx8s0/UTKFace.tar.gz'
  fpath = 'UTKFace.tar.gz'
  filename = fpath.split('/')[-1]
  if not os.path.isfile(fpath):
    os.system(f'wget {url}')
    os.system(f'tar xzf {filename}')
  else:
    print("Data already present")
    

def copy_imgs_young(fpath, data_path):
  low = 18
  high = 25
  final_path_train = data_path + "/trainA"
  final_path_test = data_path + "/testA"
  if not os.path.isdir(final_path_train):
    os.makedirs(final_path_train)
  if not os.path.isdir(final_path_test):
    os.makedirs(final_path_test)
  allowed_files = []
  for fname in tqdm(glob(fpath+"/*.jpg")):
    age = int(fname.split('/')[-1].split('_')[0])
    if age >= low and age <= high:
      allowed_files.append(fname)
      
  allowed_files.sort()
  train_len = int(0.8*len(allowed_files))
  for fname in tqdm(allowed_files[:train_len]):
      os.system(f'cp {fname} {final_path_train}/')
  for fname in tqdm(allowed_files[train_len:]):
      os.system(f'cp {fname} {final_path_test}/')
      
def copy_imgs_old(fpath, data_path):
  low = 50
  high = 60
  final_path_train = data_path + "/trainB"
  final_path_test = data_path + "/testB"
  if not os.path.isdir(final_path_train):
    os.makedirs(final_path_train)
  if not os.path.isdir(final_path_test):
    os.makedirs(final_path_test)
  allowed_files = []
  
  for fname in tqdm(glob(fpath+"/*.jpg")):
    age = int(fname.split('/')[-1].split('_')[0])
    if age >= low and age <= high:
      allowed_files.append(fname)
  
  allowed_files.sort()
  train_len = int(0.8*len(allowed_files))
  for fname in tqdm(allowed_files[:train_len]):
      os.system(f'cp {fname} {final_path_train}/')
  for fname in tqdm(allowed_files[train_len:]):
      os.system(f'cp {fname} {final_path_test}/')
      
def prepare_dataset():
  download()
  copy_imgs_young('UTKFace', 'datasets/faces')
  copy_imgs_old('UTKFace', 'datasets/faces')
  
  
prepare_dataset()

100%|██████████| 519/519 [00:05<00:00, 94.91it/s]


## Model declaration

In [0]:
class DataLoader():
  def __init__(self, dataset_name, img_res=(256, 256)):
    self.dataset_name = dataset_name
    self.img_res = img_res
    self.dataset_root_path = "datasets"

  def load_data(self, domain, batch_size=1, is_testing=False):
    data_type = "train%s" % domain if not is_testing else "test%s" % domain
    path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

    batch_images = np.random.choice(path, size=batch_size)

    imgs = []
    for img_path in batch_images:
      img = self.imread(img_path)
      if not is_testing:
        img = cv2.resize(img, self.img_res)

        if np.random.random() > 0.5:
          img = np.fliplr(img)
      else:
        img = cv2.resize(img, self.img_res)
      imgs.append(img)

    imgs = np.array(imgs)/127.5 - 1.

    return imgs

  def load_batch(self, batch_size=1, is_testing=False):
    data_type = "train" if not is_testing else "val"
    path_A = glob(f'./{self.dataset_root_path}/%s/%sA/*' % (self.dataset_name, data_type))
    path_B = glob(f'./{self.dataset_root_path}/%s/%sB/*' % (self.dataset_name, data_type))

    self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
    total_samples = self.n_batches * batch_size

    # Sample n_batches * batch_size from each path list so that model sees all
    # samples from both domains
    path_A = np.random.choice(path_A, total_samples, replace=False)
    path_B = np.random.choice(path_B, total_samples, replace=False)

    for i in range(self.n_batches-1):
      batch_A = path_A[i*batch_size:(i+1)*batch_size]
      batch_B = path_B[i*batch_size:(i+1)*batch_size]
      imgs_A, imgs_B = [], []
      for img_A, img_B in zip(batch_A, batch_B):
        img_A = self.imread(img_A)
        img_B = self.imread(img_B)

        img_A = cv2.resize(img_A, self.img_res)
        img_B = cv2.resize(img_B, self.img_res)

        if not is_testing and np.random.random() > 0.5:
            img_A = np.fliplr(img_A)
            img_B = np.fliplr(img_B)

        imgs_A.append(img_A)
        imgs_B.append(img_B)

      imgs_A = np.array(imgs_A)/127.5 - 1.
      imgs_B = np.array(imgs_B)/127.5 - 1.

      yield imgs_A, imgs_B

  def load_img(self, path):
    img = self.imread(path)
    img = cv2.resize(img, self.img_res)
    img = img/127.5 - 1.
    return img[np.newaxis, :, :, :]

  def imread(self, path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float)
    return img

In [0]:
class CycleGAN():
  def __init__(self, save_path='train', output_img_path='/Drive/My Drive/facegan/images', combined_weights_path=None):
    # Input shape
    self.img_rows = 256
    self.img_cols = 256
    self.channels = 3
    self.img_shape = (self.img_rows, self.img_cols, self.channels)

    # Configure data loader
    self.dataset_name = 'faces'
    self.data_loader = DataLoader(dataset_name=self.dataset_name,
                    img_res=(self.img_rows, self.img_cols))
    
    self.save_path = save_path
    self.output_img_path = output_img_path


    # Calculate output shape of D (PatchGAN)
    patch = int(self.img_rows / 2**4)
    patch *= 2  # hack to get the shape 32,32,1. todo: check
    self.disc_patch = (patch, patch, 1)

    # Number of filters in the first layer of G and D
    self.gf = 64
    self.df = 64

    # Loss weights
    self.lambda_cycle = 10.0          # Cycle-consistency loss
    self.lambda_id = 0.1 * self.lambda_cycle  # Identity loss

    self.lr_init = 0.0002
    self.epoch_step = 100
    
    optimizer = Adam(self.lr_init, 0.5)

    # Build and compile the discriminators
    self.d_A = self.build_discriminator()
    self.d_B = self.build_discriminator()
    print("Summary Discriminator:")
    print(self.d_A.summary())
    self.d_A.compile(loss='mse',
      optimizer=optimizer,
      metrics=['accuracy'])
    self.d_B.compile(loss='mse',
      optimizer=optimizer,
      metrics=['accuracy'])

    #-------------------------
    # Construct Computational
    #   Graph of Generators
    #-------------------------

    # Build the generators
    self.g_AB = self.build_generator()
    self.g_BA = self.build_generator()
    print("Summary Generator:")
    print(self.g_AB.summary())

    # Input images from both domains
    img_A = Input(shape=self.img_shape)
    img_B = Input(shape=self.img_shape)

    # Translate images to the other domain
    fake_B = self.g_AB(img_A)
    fake_A = self.g_BA(img_B)
    # Translate images back to original domain
    reconstr_A = self.g_BA(fake_B)
    reconstr_B = self.g_AB(fake_A)
    # Identity mapping of images
    img_A_id = self.g_BA(img_A)
    img_B_id = self.g_AB(img_B)

    # For the combined model we will only train the generators
    self.d_A.trainable = False
    self.d_B.trainable = False

    # Discriminators determines validity of translated images
    valid_A = self.d_A(fake_A)
    valid_B = self.d_B(fake_B)

    # Combined model trains generators to fool discriminators
    self.combined = Model(inputs=[img_A, img_B],
                outputs=[ valid_A, valid_B,
                    reconstr_A, reconstr_B,
                    img_A_id, img_B_id ])
    
    if combined_weights_path is not None:
        self.combined.load_weights(combined_weights_path)
    
    self.combined.compile(loss=['mse', 'mse',
                  'mae', 'mae',
                  'mae', 'mae'],
              loss_weights=[  1, 1,
                      self.lambda_cycle, self.lambda_cycle,
                      self.lambda_id, self.lambda_id ],
              optimizer=optimizer)

  def build_generator(self, name="generator"):
    gf_dim=self.gf
    output_c_dim=self.channels
    def conv2D(input_,
               output_dim, 
               ks=4, 
               s=2, 
               stddev=0.02, 
               padding='SAME', 
               activation=False, 
               instance_norm=False, 
               name="conv2d", 
               act_name="relu",
               activation_tanh=True,
               inst_norm_name="bn"):

      if activation_tanh:
        x = Conv2D(
          output_dim, 
          kernel_size=(ks, ks), 
          strides=s, 
          padding=padding, 
          kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
          name=name,
          activation='tanh',
          use_bias=False)(input_)
      elif activation:
        x = Conv2D(
            output_dim, 
            kernel_size=(ks, ks), 
            strides=s, 
            padding=padding, 
            kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
            name=name,
            activation='relu',
            use_bias=False)(input_)
      else:
        x = Conv2D(
            output_dim, 
            kernel_size=(ks, ks), 
            strides=s, 
            padding=padding, 
            kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
            name=name,
            use_bias=False)(input_)
      if instance_norm:
        x = InstanceNormalization(name=inst_norm_name)(x)

      return x

    def deconv2d(input_, output_dim, ks=4, s=2, stddev=0.02, name="deconv2d"):
      x = Conv2DTranspose(
          output_dim, 
          kernel_size=(ks, ks), 
          strides=s, 
          padding='SAME', 
          kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
          name=name,
          use_bias=False)(input_)

      return x

    def residule_block(x, dim, ks=3, s=1, name='res'):
        p = int((ks - 1) / 2)
        y = Lambda(lambda lx: tf.pad(lx, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT") )(x)
        y = conv2D(y,dim,ks=ks,s=s,padding="VALID",name=name+"_c1",activation=True,instance_norm=True,inst_norm_name=name+"_bn1")
        y = Lambda(lambda x: tf.pad(x, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT"))(y)
        y = conv2D(y,dim,ks=ks,s=s,padding="VALID",name=name+"_c2",activation=False,instance_norm=True,inst_norm_name=name+"_bn2")
        a = Add()([y,x])
        return a           # 1/2을 안해도 되나??? 뒤에 노말라이즈해서 상관없는건가....

    # Justin Johnson's model from https://github.com/jcjohnson/fast-neural-style/
    # The network with 9 blocks consists of: c7s1-32, d64, d128, R128, R128, R128,
    # R128, R128, R128, R128, R128, R128, u64, u32, c7s1-3
    
    inp = Input(shape=self.img_shape)

    c0 = Lambda(lambda x: tf.pad(x, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT") )(inp)


    c1 = conv2D(c0,gf_dim,ks=7,s=1,padding="VALID",name="g_e1_c",activation=True,instance_norm=True,inst_norm_name=name+"g_e1_bn", act_name="g_e1_relu")
    c2 = conv2D(c1,gf_dim*2,ks=3,s=2,name="g_e2_c",activation=True,instance_norm=True,inst_norm_name=name+"g_e2_bn", act_name="g_e2_relu")
    c3 = conv2D(c2,gf_dim*4,ks=3,s=2,name="g_e3_c",activation=True,instance_norm=True,inst_norm_name=name+"g_e3_bn", act_name="g_e3_relu")  # 64x64

    # define G network with 9 resnet blocks
    # 총 cnn 18층.
    r1 = residule_block(c3, gf_dim*4, name='g_r1')
    r2 = residule_block(r1, gf_dim*4, name='g_r2')
    r3 = residule_block(r2, gf_dim*4, name='g_r3')
    r4 = residule_block(r3, gf_dim*4, name='g_r4')
    r5 = residule_block(r4, gf_dim*4, name='g_r5')
    r6 = residule_block(r5, gf_dim*4, name='g_r6')
    r7 = residule_block(r6, gf_dim*4, name='g_r7')
    r8 = residule_block(r7, gf_dim*4, name='g_r8')
    r9 = residule_block(r8, gf_dim*4, name='g_r9')

    d1 = deconv2d(r9, gf_dim*2, 3, 2, name='g_d1_dc')
    d1 = InstanceNormalization(name='g_d1_bn')(d1)
    d1 = Lambda(lambda x: relu(x))(d1)

    d2 = deconv2d(d1, gf_dim, 3, 2, name='g_d2_dc')
    d2 = InstanceNormalization(name='g_d2_bn')(d2)
    d2 = Lambda(lambda x: relu(x))(d2)

    d2 = Lambda(lambda x: tf.pad(x, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT"))(d2)

    pred = conv2D(d2,output_c_dim,ks=7,s=1,padding="VALID",name="g_pred_c",activation=False,instance_norm=False,activation_tanh=True)

    return Model(inp, pred)

  def build_discriminator(self, name="discriminator"):
    df_dim=self.df
    def conv2D(input_,
               output_dim, 
               ks=4, 
               s=2, 
               stddev=0.02, 
               padding='SAME', 
               activation=False, 
               instance_norm=False, 
               name="conv2d", 
               act_name="lrelu",
               inst_norm_name="bn"):

      x = Conv2D(
          output_dim, 
          kernel_size=(ks, ks), 
          strides=s, 
          padding=padding, 
          kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
          name=name,
          use_bias=False)(input_)
      if activation:
        x = LeakyReLU(alpha=0.2,name=act_name)(x)
      if instance_norm:
        x = InstanceNormalization(name=inst_norm_name)(x)

      return x

    inp = Input(shape=self.img_shape)
    h0 = conv2D(inp, df_dim, name='d_h0_conv',activation=True)
    # h0 = lrelu(conv2d(image, df_dim, name='d_h0_conv'))
    # h0 is (128 x 128 x self.df_dim)

    h1 = conv2D(h0, df_dim*2, name='d_h1_conv',activation=True, instance_norm=True, inst_norm_name='d_bn1', act_name='d_lrelu1')
    # h1 = lrelu(instance_norm(conv2d(h0, df_dim*2, name='d_h1_conv'), 'd_bn1'))
    # h1 is (64 x 64 x self.df_dim*2)

    h2 = conv2D(h1, df_dim*4, name='d_h2_conv',activation=True, instance_norm=True, inst_norm_name='d_bn2', act_name='d_lrelu2')
    # h2 = lrelu(instance_norm(conv2d(h1, df_dim*4, name='d_h2_conv'), 'd_bn2'))
    # h2 is (32x 32 x self.df_dim*4)

    h3 = conv2D(h2, df_dim*8, name='d_h3_conv', s=1, activation=True, instance_norm=True, inst_norm_name='d_bn3', act_name='d_lrelu3')
    # h3 = lrelu(instance_norm(conv2d(h2, df_dim*8, s=1, name='d_h3_conv'), 'd_bn3'))
    # h3 is (32 x 32 x self.df_dim*8)

    h4 = conv2D(h3, 1, name='d_h3_pred', s=1, activation=False, instance_norm=False)
    # h4 = conv2d(h3, 1, s=1, name='d_h3_pred')
    # h4 is (32 x 32 x 1)

    return Model(inp, h4)

  def train(self, epochs, batch_size=1, sample_interval=500, init_epoch=0):

    start_time = datetime.datetime.now()

    # Adversarial loss ground truths
    valid = np.ones((batch_size,) + self.disc_patch)
    fake = np.zeros((batch_size,) + self.disc_patch)

    for epoch in range(init_epoch, epochs):
      lr = self.lr_init if epoch < self.epoch_step else self.lr_init*(epochs-epoch)/(epochs-self.epoch_step)    # lr이 작아짐. 100에폭이 지나면.
      for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
                
        K.set_value(self.d_A.optimizer.lr, lr)
        K.set_value(self.d_B.optimizer.lr, lr)
        K.set_value(self.combined.optimizer.lr, lr)

        # ----------------------
        #  Train Discriminators
        # ----------------------

        # Translate images to opposite domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        
        # Train the discriminators (original images = real / translated = Fake)
        dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
        dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
        dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

        dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
        dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
        dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

        # Total disciminator loss
        d_loss = 0.5 * np.add(dA_loss, dB_loss)


        # ------------------
        #  Train Generators
        # ------------------

        # Train the generators
        g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                            [valid, valid,
                            imgs_A, imgs_B,
                            imgs_A, imgs_B])

        elapsed_time = datetime.datetime.now() - start_time

        # Plot the progress
        print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                    % ( epoch, epochs,
                                      batch_i, self.data_loader.n_batches,
                                      d_loss[0], 100*d_loss[1],
                                      g_loss[0],
                                      np.mean(g_loss[1:3]),
                                      np.mean(g_loss[3:5]),
                                      np.mean(g_loss[5:6]),
                                      elapsed_time))

        # If at save interval => save generated image samples
        if batch_i % sample_interval == 0:
          self.sample_images(epoch, batch_i)
          self.save_model(epoch)

          
  def save_model(self, epoch):
    print("Saving model now")
    if not os.path.isdir(self.save_path):
      os.makedirs(self.save_path)
    # for further training - save with
    self.combined.save_weights(self.save_path+'/%s-%d.h5' % (self.dataset_name, epoch))
    
    # reload with
    # self.combined.load_weights('train/%s-%d.h5' % (self.dataset_name, epoch))

    # to generate images - save with
    self.g_AB.save_weights(self.save_path+'/%s-AB-%d.h5' % (self.dataset_name, epoch))
    self.g_BA.save_weights(self.save_path+'/%s-BA-%d.h5' % (self.dataset_name, epoch))
    
    # load with
    # self.g_AB.load_weights('train/%s-AB-%d.h5' % (self.dataset_name, epoch))
    # self.g_BA.load_weights('train/%s-BA-%d.h5' % (self.dataset_name, epoch))
    
  def sample_images(self, epoch, batch_i):
    os.makedirs(self.output_img_path+'/%s' % self.dataset_name, exist_ok=True)
    
    print("Loading weights")
    self.g_AB.load_weights('faces-AB-100.h5')
    self.g_BA.load_weights('faces-BA-100.h5')
    print("Done loading")
    
    r, c = 2, 3

    imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
    imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

    # Demo (for GIF)
    #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
    #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

    # Translate images to the other domain
    fake_B = self.g_AB.predict(imgs_A)
    fake_A = self.g_BA.predict(imgs_B)
    # Translate back to original domain
    reconstr_A = self.g_BA.predict(fake_B)
    reconstr_B = self.g_AB.predict(fake_A)
    
    
    print(imgs_A.shape, fake_B.shape, fake_A.shape)

    gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
      for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt])
        axs[i, j].set_title(titles[j])
        axs[i,j].axis('off')
        cnt += 1
    fig.savefig(self.output_img_path+"/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
    plt.close()

## Training

In [0]:
output_img_path = '/Drive/My Drive/facegan-final'
if not os.path.isdir(output_img_path):
  os.makedirs(output_img_path)
gan = CycleGAN(output_img_path=output_img_path)
# gan.train(1)


Summary Discriminator:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 256, 256, 3)       0         
_________________________________________________________________
d_h0_conv (Conv2D)           (None, 128, 128, 64)      3072      
_________________________________________________________________
lrelu (LeakyReLU)            (None, 128, 128, 64)      0         
_________________________________________________________________
d_h1_conv (Conv2D)           (None, 64, 64, 128)       131072    
_________________________________________________________________
d_lrelu1 (LeakyReLU)         (None, 64, 64, 128)       0         
_________________________________________________________________
d_bn1 (InstanceNormalization (None, 64, 64, 128)       2         
_________________________________________________________________
d_h2_conv (Conv2D)           (None, 32, 32, 256)     